In [ ]:
import mysql.connector as connector
from mysql.connector import Error

In [ ]:
# Establishing connection
try:
    connection=connector.connect(user="root",password="")
except Error as er:
    print(er.msg)

print("MySQL connection established.")

In [ ]:
# Creating a cursor object to execute SQL queries
cursor = connection.cursor()
print("Cursor object created.")

In [ ]:
cursor.execute("USE LittleLemonDB")
connection.database
print("LittleLemonDB Database selected.")

In [ ]:
show_tables_query = "SHOW TABLES;" 

cursor.execute(show_tables_query)
results=cursor.fetchall()
columns=cursor.column_names

print(columns)
for result in results:
    print(result)

In [ ]:
# implementing GetMaxQuantity
max_quantity_query = """
DROP PROCEDURE IF EXISTS GetMaxQuantity;
DELIMITER //

CREATE PROCEDURE GetMaxQuantity()
BEGIN
SELECT MAX(quantity) AS 'Max Quantity in Order'
FROM Orders;
END; //

DELIMITER ;"""

cursor.execute(max_quantity_query)
print("GetMaxQuantity procedure created.")

In [ ]:
get_order_detail_query = """
PREPARE GetOrderDetail FROM
'SELECT OrderID, Quantity, TotalCost AS Cost
FROM orders
WHERE CustomerID = ?';

SET @id = 1;
EXECUTE GetOrderDetail USING @id;
"""
cursor.execute(get_order_detail_query)
print("GetOrderDetail procedure created.")

In [ ]:
cancel_booking_query = """
DELIMITER //

CREATE PROCEDURE CancelBooking(IN Booking_id INT)
BEGIN
    DELETE FROM Bookings WHERE BookingID = Booking_id;
    SELECT CONCAT("Booking ", Booking_id, " cancelled") AS "Confirmation";
END //
DELIMITER ;
"""
cursor.execute(cancel_booking_query)
print("CancelBooking procedure created.")

In [ ]:
add_booking_query = """
DELIMITER //
CREATE PROCEDURE IF NOT EXISTS AddBooking (table_nb INT, staff_id INT, book_date DATE, cust_id CHAR(11), book_time TIME)
BEGIN
START TRANSACTION;
	SELECT 1 AS 'status'
	FROM bookings 
	WHERE booking_date = book_date
	AND table_no = table_nb
	AND booking_time = book_time
	LIMIT 1
	INTO @booking_status;

IF @booking_status = 1 THEN
	ROLLBACK;
	SELECT CONCAT("A client has already booked table ", table_nb, " on ", book_date, " at ", book_time, ". Cannot add booking. Transaction cancelled.") AS 'Booking status';

ELSE
	INSERT INTO bookings (table_no, booking_staff_id, booking_date, booking_customer_id, booking_time ) VALUES 
	(table_nb, staff_id, book_date, cust_id, book_time);
COMMIT;

SELECT CONCAT("Table ", table_number, " is free - booking successful.") AS 'Booking status';
END IF;
END //

DELIMITER ;
"""
cursor.execute(add_booking_query)
print("AddBooking procedure created.")

In [ ]:
cancel_booking_query = """
DELIMITER //

CREATE PROCEDURE CancelBooking(IN Booking_id INT)
BEGIN
    DELETE FROM Bookings WHERE BookingID = Booking_id;
    SELECT CONCAT("Booking ", Booking_id, " cancelled") AS "Confirmation";
END //
DELIMITER ;
"""
cursor.execute(cancel_booking_query)
print("CancelBooking procedure created.")

In [ ]:
check_booking_query = """
DELIMITER //
CREATE PROCEDURE IF NOT EXISTS CheckBooking (book_date DATE, book_time TIME, table_num INT)
BEGIN
DECLARE table_booked INT DEFAULT 0;
SELECT 
    COUNT(table_booked)
INTO table_booked FROM
    bookings
WHERE
    booking_date = book_date
        AND booking_time = book_time
        AND table_no = table_num;
    IF table_booked > 0 THEN
      SELECT CONCAT( "Table ",table_num, " is already booked at ",book_time, " on ", book_date,".") AS "Booking status";
      ELSE 
      SELECT CONCAT( "Table ",table_num," is not booked at ",book_time," on ",book_date,".") AS "Booking status";
    END IF;
END//
DELIMITER ;
"""
cursor.execute(check_booking_query)
print("CheckBooking procedure created.")

In [ ]:
update_booking_query = """
DELIMITER //

CREATE PROCEDURE UpdateBooking(IN Booking_id INT, IN Booking_Date DATETIME)
BEGIN
UPDATE Bookings SET BookingDate = Booking_Date
WHERE BookingID = Booking_id;
SELECT CONCAT("Booking ", Booking_id, " updated") AS "Confirmation";
END //
DELIMITER ;
"""
cursor.execute(update_booking_query)
print("UpdateBooking procedure created.")